In [2]:
# importing required libraries
from tensorflow import keras
import os 
from pathlib import Path

In [34]:
# downloading the ImageNet model with output layer removed and input shape (224, 224, 3)
base_model = keras.applications.VGG16(
    weights = 'imagenet',
    input_shape = (224, 224, 3),
    include_top = False)
 
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [35]:
# Freeze base model
base_model.trainable = False

In [36]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


In [37]:
# compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
# Augment the Data 
datagen = keras.preprocessing.image.ImageDataGenerator(
        samplewise_center = True,  # set each sample mean to 0
        rotation_range = 10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range = 0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range = 0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip = True)

In [39]:
# load and iterate training dataset
dirPath = os.path.join(Path(os.path.realpath("__file__")).parent.parent, "assets/data")
train_it = datagen.flow_from_directory(dirPath + "/train", 
                                       target_size = (224, 224), 
                                       color_mode = 'rgb', 
                                       class_mode = "categorical",
                                       batch_size = 8)
# load and iterate validation dataset
valid_it = datagen.flow_from_directory(dirPath + "/valid", 
                                       target_size = (224, 224), 
                                       color_mode = 'rgb', 
                                       class_mode = "categorical",
                                       batch_size = 8)

Found 1320 images belonging to 6 classes.
Found 162 images belonging to 6 classes.


In [40]:
# Train the Model
model.fit(train_it,
          validation_data = valid_it,
          steps_per_epoch = train_it.samples/train_it.batch_size,
          validation_steps = valid_it.samples/valid_it.batch_size,
          epochs = 15)

Epoch 1/15
165/165 [==============================] - 277s 2s/step - loss: 1.5625 - accuracy: 0.4727 - val_loss: 0.7758 - val_accuracy: 0.6914
Epoch 2/15
165/165 [==============================] - 297s 2s/step - loss: 0.7354 - accuracy: 0.7500 - val_loss: 0.5593 - val_accuracy: 0.8025
Epoch 3/15
165/165 [==============================] - 286s 2s/step - loss: 0.4640 - accuracy: 0.8273 - val_loss: 0.5164 - val_accuracy: 0.7840
Epoch 4/15
165/165 [==============================] - 287s 2s/step - loss: 0.3569 - accuracy: 0.8705 - val_loss: 0.3721 - val_accuracy: 0.8765
Epoch 5/15
165/165 [==============================] - 288s 2s/step - loss: 0.3107 - accuracy: 0.8833 - val_loss: 0.3600 - val_accuracy: 0.8704
Epoch 6/15
165/165 [==============================] - 300s 2s/step - loss: 0.2433 - accuracy: 0.9152 - val_loss: 0.2955 - val_accuracy: 0.8951
Epoch 7/15
165/165 [==============================] - 302s 2s/step - loss: 0.2134 - accuracy: 0.9189 - val_loss: 0.2475 - val_accuracy: 0.9012

In [41]:
# # Unfreeze the base model
# base_model.trainable = True

# # Compile the model with a low learning rate
# model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = 0.0001),
#               loss='categorical_crossentropy', metrics=['accuracy'])

In [42]:
# model.fit(train_it,
#           validation_data = valid_it,
#           steps_per_epoch = train_it.samples/train_it.batch_size,
#           validation_steps = valid_it.samples/valid_it.batch_size,
#           epochs = 8)

In [47]:
# saving the trained model
dirPath = os.path.join(Path(os.path.realpath("__file__")).parent.parent, "assets/model")
if not os.path.exists(dirPath): os.makedirs(dirPath)
    
model.save(dirPath + "/hand_gesture_classify_model.h5")